In [ ]:
import argparse
import base64
import json

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
import time
from PIL import Image
from PIL import ImageOps
from flask import Flask, render_template
from io import BytesIO

from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

import cv2

sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None
target_shape = (80, 40) #original size 320, 160

@sio.on('telemetry')
def telemetry(sid, data):
    # The current steering angle of the car
    steering_angle = data["steering_angle"]
    # The current throttle of the car
    throttle = data["throttle"]
    # The current speed of the car
    speed = np.array([float(data["speed"])])
    # The current image from the center camera of the car
    imgString = data["image"]
    image = Image.open(BytesIO(base64.b64decode(imgString)))
    image_array = np.asarray(image)
    image_array = cv2.resize(image_array, target_shape)
    transformed_image_array = image_array[None, :, :, :]
    transformed_image_array = preprocess_input(transformed_image_array.astype('float'))
    # This model currently assumes that the features of the model are just the images. Feel free to change this.
    #print('predicting')
    steering_angle = float(loaded_model.predict(transformed_image_array, batch_size=1))
    # The driving model currently just outputs a constant throttle. Feel free to edit this.
    throttle = 0.3
    print(steering_angle, throttle)
    send_control(steering_angle, throttle)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit("steer", data={
    'steering_angle': steering_angle.__str__(),
    'throttle': throttle.__str__()
    }, skip_sid=True)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Remote Driving')
    parser.add_argument('model', type=str,
    help='Path to model definition json. Model weights should be on the same path.')
    #args = parser.parse_args()

    model_file = 'vgg16-pretrained-gen-0.0376.json'
    with open(model_file, 'r') as jfile:
        loaded_model = model_from_json(json.load(jfile))
    loaded_model.summary()
    weights_file = model_file.replace('json', 'h5')
    loaded_model.load_weights(weights_file)

    loaded_model.compile("adam", "mse")

    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, None, None, 64 1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, None, None, 64 36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, None, None, 64 0           block1_conv2[0][0]               
___________________________________________________________________________________________

(17796) wsgi starting up on http://0.0.0.0:4567
(17796) accepted ('50.174.21.68', 56610)


connect  ed0854e7752b46cea8cb9e844a774891
-0.01182015985250473 0.3
-0.011857867240905762 0.3
-0.011857867240905762 0.3
-0.011858909390866756 0.3
-0.011976911686360836 0.3
-0.011976911686360836 0.3
-0.012303936295211315 0.3
-0.012265284545719624 0.3
-0.012241274118423462 0.3
-0.012241274118423462 0.3
-0.012696265242993832 0.3
-0.012728209607303143 0.3
-0.012385223992168903 0.3
-0.012385223992168903 0.3
-0.013223554007709026 0.3
-0.012834220193326473 0.3
-0.012775578536093235 0.3
-0.012775578536093235 0.3
-0.012762541882693768 0.3
-0.01245263870805502 0.3
-0.011952148750424385 0.3
-0.011952148750424385 0.3
-0.013129930943250656 0.3
-0.013780380599200726 0.3
-0.01390878576785326 0.3
-0.012405245564877987 0.3
-0.012405245564877987 0.3
-0.013108720071613789 0.3
-0.012728999368846416 0.3
-0.012728999368846416 0.3
-0.012345348484814167 0.3
-0.012786498293280602 0.3
-0.012786498293280602 0.3
-0.012571977451443672 0.3
-0.011999567039310932 0.3
-0.012474467977881432 0.3
-0.012474467977881432 0.3